# seq2seq model

In [1]:
import random
import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(42)

In [2]:
#Add raw data
raw = ["I feel hungry.	나는 배가 고프다.",
       "Pytorch is very easy.	파이토치는 매우 쉽다.",
       "Pytorch is a framework for deep learning.	파이토치는 딥러닝을 위한 프레임워크이다.",
       "Pytorch is very clear to use.	파이토치는 사용하기 매우 직관적이다."]

In [3]:
SOS_token = 0
EOS_token = 1

In [4]:
class Vocab:
    def __init__(self):
        self.vocab2index = {"<SOS>": SOS_token, "<EOS>":EOS_token}
        self.index2vocab = {SOS_token: "<SOS>", EOS_token: "<EOS>"}
        self.vocab_count = {}
        self.n_vocab = len(self.vocab2index)
    def add_vocab(self, sentence):
        for word in sentence.split():
            if word not in self.vocab2index:
                self.vocab2index[word] = self.n_vocab
                self.vocab_count[word] = 1
                self.index2vocab[self.n_vocab] = word
                self.n_vocab += 1
            else:
                self.vocab_count[word] +=1

In [5]:
def filter_pair(pair, source_max_length, target_max_length):
    return len(pair[0].split(" ")) < source_max_length and len(pair[1].split(" ")) < target_max_length

In [9]:
def preprocess(corpus, source_max_length, target_max_length):
    print("Reading corpus...")
    pairs = []
    for line in corpus:
        pairs.append([s for s in line.strip().lower().split("\t")])
    print("Read {} setence pairs".format(len(pairs)))

    pairs = [pair for pair in pairs if filter_pair(pair, source_max_length=source_max_length, target_max_length=target_max_length)]
    print("Accepted sentence pairs: {}".format(len(pairs)))

    source_vocab = Vocab()
    target_vocab = Vocab()

    print("Counting Words...")

    for pair in pairs:
        source_vocab.add_vocab(pair[0])
        target_vocab.add_vocab(pair[1])
    print("source vocab size:".format(source_vocab.n_vocab))
    print("target vocab size:".format(target_vocab.n_vocab))

In [10]:
class Encoder(nn.Module):
    def __init__(self, input_size ,hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        return x, hidden

In [11]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden):
        x = self.embedding(x).view(1,1,-1)
        x, hidden = self.gru(x, hidden)
        x = self.softmax(self.fc(x))
        return x, hidden

In [12]:
def tensorize(vocab, sentence):
    indexes = [vocab.vocab2index[word] for word in sentence.split()]
    indexes.append(vocab.vocab2index["<EOS>"])
    return torch.Tensor(indexes).long().view(-1, 1)